# Problem Statement: 
### Using ML/DL techniques, match similar products from the Flipkart dataset with the Amazon dataset. Once similar products are matched, display the retail price from FK and AMZ side by side. Please explore as many techniques as possible before choosing the final technique. You may either display the final result in single table format OR You may create a simple form where we input the product name and the output of prices of the product from both websites are displayed.

## 1. DATA UNDERSTANDING

In [1]:
# Import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import fuzzymatcher

import warnings
warnings.filterwarnings('ignore')

### Read the Amazon Dataset

In [2]:
#Import and read the dataset

amazon_dataset = pd.read_csv("amz_com-ecommerce_sample.csv", encoding= 'unicode_escape')
amazon_dataset.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

In [3]:
amazon_dataset.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
amazon_dataset.shape

(20000, 15)

In [5]:
amazon_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   uniq_id                  20000 non-null  object
 1   crawl_timestamp          20000 non-null  object
 2   product_url              20000 non-null  object
 3   product_name             20000 non-null  object
 4   product_category_tree    20000 non-null  object
 5   pid                      20000 non-null  object
 6   retail_price             20000 non-null  int64 
 7   discounted_price         20000 non-null  int64 
 8   image                    19997 non-null  object
 9   is_FK_Advantage_product  20000 non-null  bool  
 10  description              19998 non-null  object
 11  product_rating           20000 non-null  object
 12  overall_rating           20000 non-null  object
 13  brand                    14136 non-null  object
 14  product_specifications   19986 non-nul

In [6]:
amazon_dataset.describe()

,retail_price,discounted_price
count,20000.000000,20000.00000
mean,2957.095150,2364.59705
std,8993.993257,8994.62368
min,-20.000000,0.00000
25%,647.000000,424.00000
50%,999.000000,663.00000
75%,1986.000000,1235.00000
max,571223.000000,726879.00000


In [7]:
# Checking information about data.

def metadata_matrix(data) : 
    return pd.DataFrame({
                'Datatype' : data.dtypes.astype(str), 
                'Non_Null_Count': data.count(axis = 0).astype(int), 
                'Null_Count': data.isnull().sum().astype(int), 
                'Null_Percentage': round(data.isnull().sum()/len(data) * 100 , 2).sort_values(ascending=False), 
                'Unique_Values_Count': data.nunique().astype(int) 
                 }).sort_values(by='Null_Percentage', ascending=False)

In [8]:
metadata_matrix(amazon_dataset)

,Datatype,Non_Null_Count,Null_Count,Null_Percentage,Unique_Values_Count
brand,object,14136,5864,29.32,3499
product_specifications,object,19986,14,0.07,18825
image,object,19997,3,0.02,18589
description,object,19998,2,0.01,17540
crawl_timestamp,object,20000,0,0.00,371
discounted_price,int64,20000,0,0.00,3993
is_FK_Advantage_product,bool,20000,0,0.00,2
overall_rating,object,20000,0,0.00,36
pid,object,20000,0,0.00,19998
product_category_tree,object,20000,0,0.00,6466


### Read the Flipkart Dataset

In [9]:
#Import and read the dataset

flipkart_dataset = pd.read_csv("flipkart_com-ecommerce_sample.csv")
flipkart_dataset.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

In [10]:
flipkart_dataset.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [11]:
flipkart_dataset.shape

(20000, 15)

In [12]:
flipkart_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [13]:
flipkart_dataset.describe()

,retail_price,discounted_price
count,19922.000000,19922.000000
mean,2979.206104,1973.401767
std,9009.639341,7333.586040
min,35.000000,35.000000
25%,666.000000,350.000000
50%,1040.000000,550.000000
75%,1999.000000,999.000000
max,571230.000000,571230.000000


In [14]:
metadata_matrix(flipkart_dataset)

,Datatype,Non_Null_Count,Null_Count,Null_Percentage,Unique_Values_Count
brand,object,14136,5864,29.32,3499
discounted_price,float64,19922,78,0.39,2448
retail_price,float64,19922,78,0.39,2247
product_specifications,object,19986,14,0.07,18825
image,object,19997,3,0.02,18589
description,object,19998,2,0.01,17539
crawl_timestamp,object,20000,0,0.00,371
is_FK_Advantage_product,bool,20000,0,0.00,2
overall_rating,object,20000,0,0.00,36
pid,object,20000,0,0.00,19998


## 2. Data Cleaning

In [15]:
# Dropping "Brand"-column from both the dataset because for the analysis, it is of no use

amazon_dataset = amazon_dataset.drop(columns=['brand'])

flipkart_dataset = flipkart_dataset.drop(columns=['brand'])


In [16]:
# Check for missing values percentage in Product_rating column of Amamzon Dataset

(amazon_dataset['product_rating'] == 'No rating available').sum()/amazon_dataset['product_rating'].shape[0]*100

90.755

In [17]:
# 90.75% of the values in Product_rating is "No rating available" which means these are null values
# Hence remove Product_rating column of Amamzon Dataset

amazon_dataset.drop(columns=['product_rating'], inplace=True)

In [18]:
# Check for missing values percentage in Product_rating column of Flipkart dataset

(flipkart_dataset['product_rating'] == 'No rating available').sum()/flipkart_dataset['product_rating'].shape[0]*100

90.755

In [19]:
# 90.75% of the values in Product_rating is "No rating available" which means these are null values
# Hence remove Product_rating column of Flipkart Dataset

flipkart_dataset.drop(columns=['product_rating'], inplace=True)

In [20]:
# Dropping columns from Amazon & Flipkart dataset as these are redundant for analysis


amazon_dataset = amazon_dataset.drop(columns=['crawl_timestamp', 'product_url','product_category_tree', 'pid','image',
                                 'is_FK_Advantage_product','overall_rating','description','product_specifications'])

flipkart_dataset = flipkart_dataset.drop(columns=['crawl_timestamp', 'product_url','product_category_tree', 'pid','image',
                                 'is_FK_Advantage_product','overall_rating','description','product_specifications'])


In [21]:
amazon_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   uniq_id           20000 non-null  object
 1   product_name      20000 non-null  object
 2   retail_price      20000 non-null  int64 
 3   discounted_price  20000 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 625.1+ KB


In [22]:
flipkart_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uniq_id           20000 non-null  object 
 1   product_name      20000 non-null  object 
 2   retail_price      19922 non-null  float64
 3   discounted_price  19922 non-null  float64
dtypes: float64(2), object(2)
memory usage: 625.1+ KB


## 3. Merging Flipkart & Amazon dataset

In [23]:
# Merging the both datasets

amz_fk_dataset = fuzzymatcher.fuzzy_left_join( flipkart_dataset, amazon_dataset, 
                                              left_on = 'product_name', 
                                              right_on = 'product_name')

In [24]:
amz_fk_dataset.head()

,best_match_score,__id_left,__id_right,uniq_id_left,product_name_left,retail_price_left,discounted_price_left,uniq_id_right,product_name_right,retail_price_right,discounted_price_right
0,1.104828,0_left,0_right,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,999.0,379.0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,982,438
7,1.101738,1_left,1_right,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,32143,29121
11,0.511402,2_left,2_right,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,999.0,499.0,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,991,551
12,1.104828,3_left,0_right,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,699.0,267.0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,982,438
19,1.472761,4_left,4_right,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,208,258


In [25]:
amz_fk_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 143824
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   best_match_score        20000 non-null  float64
 1   __id_left               20000 non-null  object 
 2   __id_right              20000 non-null  object 
 3   uniq_id_left            20000 non-null  object 
 4   product_name_left       20000 non-null  object 
 5   retail_price_left       19922 non-null  float64
 6   discounted_price_left   19922 non-null  float64
 7   uniq_id_right           20000 non-null  object 
 8   product_name_right      20000 non-null  object 
 9   retail_price_right      20000 non-null  int64  
 10  discounted_price_right  20000 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 1.8+ MB


In [28]:
# Let's rename the common columns for better readability and understanding

amz_fk_dataset = amz_fk_dataset.rename(columns={'product_name_left':'Product name in Flipkart',
                                                'retail_price_left':'Retail Price in Flipkart',
                                                'discounted_price_left':'Discounted Price in Flipkart',
                                                'product_name_right':'Product Name in Amazon',
                                                'retail_price_right':'Retail Price in Amazon',
                                                'discounted_price_right':'Discounted Price in Amazon'})


In [29]:
amz_fk_dataset.columns

Index(['best_match_score', '__id_left', '__id_right', 'uniq_id',
       'Product name in Flipkart', 'Retail Price in Flipkart',
       'Discounted Price in Flipkart', 'uniq_id_right',
       'Product Name in Amazon', 'Retail Price in Amazon',
       'Discounted Price in Amazon'],
      dtype='object')

In [30]:
amz_fk_dataset = amz_fk_dataset.drop(columns=['best_match_score','__id_left','__id_right','uniq_id_left',
                                             'uniq_id_right'])

In [31]:
metadata_matrix(amz_fk_dataset)

,Datatype,Non_Null_Count,Null_Count,Null_Percentage,Unique_Values_Count
Discounted Price in Flipkart,float64,19922,78,0.39,2448
Retail Price in Flipkart,float64,19922,78,0.39,2247
Discounted Price in Amazon,int64,20000,0,0.00,3447
Product Name in Amazon,object,20000,0,0.00,12565
Product name in Flipkart,object,20000,0,0.00,12676
Retail Price in Amazon,int64,20000,0,0.00,3122
uniq_id,object,20000,0,0.00,20000


In [32]:
# Dropping the null values, as they've been created by the outer join
amz_fk_dataset.dropna(inplace=True)

In [33]:
# Drop Duplicates present in the dataset
amz_fk_dataset.drop_duplicates(inplace=True)

In [34]:
metadata_matrix(amz_fk_dataset)

,Datatype,Non_Null_Count,Null_Count,Null_Percentage,Unique_Values_Count
uniq_id,object,19922,0,0.0,19922
Product name in Flipkart,object,19922,0,0.0,12626
Retail Price in Flipkart,float64,19922,0,0.0,2247
Discounted Price in Flipkart,float64,19922,0,0.0,2448
Product Name in Amazon,object,19922,0,0.0,12515
Retail Price in Amazon,int64,19922,0,0.0,3107
Discounted Price in Amazon,int64,19922,0,0.0,3447


In [38]:
amz_fk_dataset

,uniq_id,Product name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart,Product Name in Amazon,Retail Price in Amazon,Discounted Price in Amazon
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,982,438
7,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32143,29121
11,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,999.0,499.0,AW Bellies,991,551
12,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,982,438
19,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,Sicons All Purpose Arnica Dog Shampoo,208,258
...,...,...,...,...,...,...,...
143748,7179d2f6c4ad50a17d014ca1d2815156,WallDesign Small Vinyl Sticker,1500.0,730.0,WALLDESIGN SMALL VINYL STICKER,1070,662
143760,71ac419198359d37b8fe5e3fffdfee09,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0,WALLMANTRA LARGE VINYL STICKERS STICKER,1422,1484
143810,93e9d343837400ce0d7980874ece471c,Elite Collection Medium Acrylic Sticker,1299.0,999.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1290,1139
143817,669e79b8fa5d9ae020841c0c97d5e935,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1290,1139
